In [1]:
import pandas as pd
import polars as pl

In [2]:
def filter_country(df):
    return df.filter(
        pl.col('Country/region').is_in(['United States', 'Canada'])
    )


def compute_z_scores(df):
    return df.with_columns(
        z_gdp=((pl.col('gdp').log10() - 2.41108) / 0.41621).alias('z_gdp'),
        z_pop=((pl.col('population').log10() - 6.53039) / 0.33281).alias('z_pop')
    )


def compute_market(df):
    return df.with_columns(
        market=(0.5 + 0.1 * (0.35 * pl.col('z_gdp') + 0.65 * pl.col('z_pop'))).alias('market')
    )


def compute_m_plus(df):
    return df.with_columns(
        (pl.col('market') * 0.6021).alias('m_plus')
    )


def compute_tru(df):
    return df.with_columns(
        (146946 * (7.5268 * pl.col('m_plus')).exp() / 1000000).alias('tru')
    )

In [3]:
cities = (
    pl.DataFrame(pd.read_html("https://en.wikipedia.org/wiki/List_of_cities_by_GDP")[1])
    .filter(
        pl.col('Country/region').is_in(['United States', 'Canada'])
    )
    .select(
        pl.col('City proper/metropolitan area').alias('city'),
        pl.col('Country/region').alias('country'),
        pl.col('Official est. GDP up to date (billion US$)').str.split(' ').list[0].str.split(' ').list[0].str.replace(
            ',', '').cast(float).alias('gdp'),
        pl.col('Metropolitan population').str.split(' ').list[0].str.split('	').list[0].str.split('[').list[
            0].str.replace(',', '').str.replace(',', '').cast(float).alias('population')
    )
    .sort('population', descending=True)
)

In [9]:
(
    cities
    .filter(pl.col('city').str.contains('NC'))
    .to_pandas()
)

,city,country,gdp,population
0,"Charlotte-Concord-Gastonia, NC-SC MSA",United States,228.913,2756069.0
1,"Virginia Beach-Norfolk-Newport News, VA-NC MSA",United States,116.686,1806840.0
2,"Raleigh-Cary, NC MSA",United States,119.675,1484338.0
3,"Greensboro-High Point, NC MSA",United States,49.029,784101.0
4,"Winston-Salem, NC MSA",United States,40.940,688471.0
5,"Durham-Chapel Hill, NC MSA",United States,63.950,664310.0
6,"Myrtle Beach-Conway-North Myrtle Beach, SC-NC MSA",United States,25.132,536165.0
7,"Fayetteville, NC MSA",United States,25.960,529318.0
8,"Asheville, NC MSA",United States,26.228,476072.0
9,"Hickory-Lenoir-Morganton, NC MSA",United States,18.281,368347.0


In [4]:
market_maps = {
    'Anaheim Ducks': [
        {
            'market': 'Los Angeles–Long Beach, CA CSA',
            'percent': 0.3,
        },
        {
            'market': 'Riverside-San Bernardino-Ontario, CA MSA',
            'percent': 0.6,
        }
    ],
    'Arizona Coyotes': [
        {
            'market': 'Phoenix-Mesa-Chandler, AZ MSA',
            'percent': 1.0,
        },
        {
            'market': 'Tucson, AZ MSA',
            'percent': 0.5,
        },
        {
            'market': 'Flagstaff, AZ MSA',
            'percent': 0.5,
        }
    ],
    'Boston Bruins': [
        {
            'market': 'Boston-Cambridge-Newton, MA-NH MSA',
            'percent': 1.0,
        },
        {
            'market': 'Worcester, MA-CT MSA',
            'percent': 0.5,
        },
        {
            'market': 'Providence-Warwick, RI-MA MSA',
            'percent': 0.5,
        },
        {
            'market': 'Portland-South Portland, ME MSA',
            'percent': 0.5
        },
        {
            'market': 'Manchester-Nashua, NH MSA',
            'percent': 0.5,
        }
    ],
    'Buffalo Sabres': [
        {
            'market': 'Buffalo-Niagara Falls, NY MSA',
            'percent': 1.0,
        },
        {
            'market': 'Rochester, NY MSA',
            'percent': 0.5,
        },
        {
            'market': 'Syracuse, NY MSA',
            'percent': 0.5,
        },
        {
            'market': 'St. Catharines–Niagara Falls',
            'percent': 0.5,
        }
    ],
    'Calgary Flames': [
        {
            'market': 'Calgary Metropolitan Region',
            'percent': 1.0,
        },
        {
            'market': 'Lethbridge',
            'percent': 0.5,
        },
        {
            'market': 'Regina, Saskatchewan',
            'percent': 0.3,
        },
        {
            'market': 'Saskatoon',
            'percent': 0.3,
        }
    ],
    'Carolina Hurricanes': [
        {
            'market': 'Raleigh-Cary, NC MSA',
            'percent': 1.0,
        },
        {
            'market': 'Durham-Chapel Hill, NC MSA',
            'percent': 1,
        },
        {
            'market': 'Charlotte-Concord-Gastonia, NC-SC MSA',
            'percent': 0.5,
        },
        {
            'market': 'Greensboro-High Point, NC MSA',
            'percent': 0.5,
        },
        {
            'market': 'Fayetteville, NC MSA',
            'percent': 0.5,
        }
    ],
    'Chicago Blackhawks': [
        {
            'market': 'Chicago–Naperville, IL–IN–WI CSA',
            'percent': 1.0,
        },
        {
            'market': 'Milwaukee-Waukesha, WI MSA',
            'percent': 0.5,
        },
        {
            'market': 'Rockford, IL MSA',
            'percent': 0.5,
        },
        {
            'market': 'Peoria, IL MSA',
            'percent': 0.5,
        },
        {
            'market': 'South Bend-Mishawaka, IN-MI MSA',
            'percent': 0.5,
        },
    ],
    'Columbus Blue Jackets': [
        {
            'market': 'Columbus, OH MSA',
            'percent': 1.0,
        },
        {
            'market': 'Toledo, OH MSA',
            'percent': 0.5,
        },
        {
            'market': 'Cincinnati, OH-KY-IN MSA',
            'percent': 0.5,
        },
        {
            'market': 'Dayton, OH MSA',
            'percent': 0.5,
        },
    ],
    'Colorado Avalanche': [
        {
            'market': 'Denver-Aurora-Lakewood, CO MSA',
            'percent': 1.0,
        },
        {
            'market': 'Colorado Springs, CO MSA',
            'percent': 0.5,
        },
        {
            'market': 'Fort Collins, CO MSA',
            'percent': 0.5,
        },
        {
            'market': 'Boulder, CO MSA',
            'percent': 0.5,
        },
    ],
    'Dallas Stars': [
        {
            'market': 'Dallas-Fort Worth-Arlington, TX MSA',
            'percent': 1.0,
        },
        {
            'market': 'Waco, TX MSA',
            'percent': 0.5,
        },
        {
            'market': 'Sherman-Denison, TX MSA',
            'percent': 0.5,
        },
        {
            'market': 'Tyler, TX MSA',
            'percent': 0.5,
        },
    ],
    'Detroit Red Wings': [
        {
            'market': 'Detroit-Warren-Dearborn, MI Metropolitan Statistical Area',
            'percent': 1.0,
        },
        {
            'market': 'Grand Rapids-Kentwood, MI MSA',
            'percent': 0.5,
        },
        {
            'market': 'Flint, MI MSA',
            'percent': 0.5,
        },
        {
            'market': 'Ann Arbor, MI MSA',
            'percent': 0.5,
        },
        {
            'market': 'Lansing-East Lansing, MI MSA',
            'percent': 0.5,
        },
        {
            'market': 'Toledo, OH MSA',
            'percent': 0.5,
        },
    ],
    'Edmonton Oilers': [
        {
            'market': 'Edmonton Metropolitan Region',
            'percent': 1.0,
        },
        {
            'market': 'Lethbridge',
            'percent': 0.5,
        },
        {
            'market': 'Regina, Saskatchewan',
            'percent': 0.3,
        },
        {
            'market': 'Saskatoon',
            'percent': 0.3,
        }
    ],
    'Florida Panthers': [
        {
            'market': 'Miami-Fort Lauderdale-Pompano Beach, FL MSA',
            'percent': 1.0,
        },
        {
            'market': 'Cape Coral-Fort Myers, FL MSA',
            'percent': 0.5,
        },
    ],
    'Los Angeles Kings': [
        {
            'market': 'Los Angeles–Long Beach, CA CSA',
            'percent': 0.7
        },
        {
            'market': 'Riverside-San Bernardino-Ontario, CA MSA',
            'percent': 0.4
        }
    ],
    'Minnesota Wild': [
        {
            'market': 'Minneapolis-St. Paul-Bloomington, MN-WI MSA',
            'percent': 1.0,
        },
        {
            'market': 'Duluth, MN-WI MSA',
            'percent': 0.5,
        },
        {
            'market': 'Rochester, MN MSA',
            'percent': 0.5,
        },
        {
            'market': 'St. Cloud, MN MSA',
            'percent': 0.5,
        },
    ],
    'Montreal Canadiens': [
        {
            'market': 'Greater Montreal',
            'percent': 1.0,
        },
        {
            'market': 'Quebec City',
            'percent': 0.5,
        },
        {
            'market': 'Sherbrooke',
            'percent': 0.5,
        },
        {
            'market': 'Trois-Rivières',
            'percent': 0.5,
        },
    ],
    'Nashville Predators': [
        {
            'market': 'Nashville-Davidson–Murfreesboro–Franklin, TN MSA',
            'percent': 1.0,
        },
        {
            'market': 'Knoxville, TN MSA',
            'percent': 0.5,
        },
        {
            'market': 'Clarksville, TN-KY MSA',
            'percent': 0.5,
        },
        {
            'market': 'Huntsville, AL MSA',
            'percent': 0.5,
        },
    ],
    'New Jersey Devils': [
        {
            'market': 'New York–Newark, NY-NJ-CT-PA CSA',
            'percent': 0.2,
        },
        {
            'market': 'Trenton-Princeton, NJ MSA',
            'percent': 0.5,
        },
    ],
    'New York Islanders': [
        {
            'market': 'New York–Newark, NY-NJ-CT-PA CSA',
            'percent': 0.3,
        },
        {
            'market': 'Bridgeport-Stamford-Norwalk, CT MSA',
            'percent': 0.5,
        },
    ],
    'New York Rangers': [
        {
            'market': 'New York–Newark, NY-NJ-CT-PA CSA',
            'percent': 0.5,
        },
        {
            'market': 'Hartford-East Hartford-Middletown, CT MSA',
            'percent': 0.2,
        },
        {
            'market': 'Bridgeport-Stamford-Norwalk, CT MSA',
            'percent': 0.15,
        },
    ],
    'Ottawa Senators': [
        {
            'market': 'Ottawa–Gatineau',
            'percent': 1,
        },
        {
            'market': 'Kingston, Ontario',
            'percent': 0.5,
        },
    ],
    'Philadelphia Flyers': [
        {
            'market': 'Philadelphia-Camden-Wilmington, PA-NJ-DE-MD MSA',
            'percent': 1,
        },
        {
            'market': 'Allentown-Bethlehem-Easton, PA-NJ MSA',
            'percent': 0.5,
        },
    ],
    'Pittsburgh Penguins': [
        {
            'market': 'Pittsburgh, PA MSA',
            'percent': 1,
        },
        {
            'market': 'Youngstown-Warren-Boardman, OH-PA MSA',
            'percent': 0.5,
        },
        {
            'market': 'Erie, PA MSA',
            'percent': 0.5,
        },
        {
            'market': 'Scranton–Wilkes-Barre, PA MSA',
            'percent': 0.5,
        },
        {
            'market': 'Wheeling, WV-OH MSA',
            'percent': 0.5,
        },
    ],
    'San Jose Sharks': [
        {
            'market': 'San Jose-San Francisco–Oakland, CA CSA',
            'percent': 0.5,
        },
    ],
    'Seattle Kraken': [
        {
            'market': 'Seattle-Tacoma-Bellevue, WA MSA',
            'percent': 1,
        },
        {
            'market': 'Bremerton-Silverdale-Port Orchard, WA MSA',
            'percent': 0.5,
        },
        {
            'market': 'Olympia-Lacey-Tumwater, WA MSA',
            'percent': 0.5,
        },
    ],
    'St. Louis Blues': [
        {
            'market': 'St. Louis, MO-IL MSA',
            'percent': 1,
        },
        {
            'market': 'Springfield, IL MSA',
            'percent': 0.5,
        },
        {
            'market': 'Columbia, MO MSA',
            'percent': 0.5,
        },
    ],
    'Tampa Bay Lightning': [
        {
            'market': 'Tampa-St. Petersburg-Clearwater, FL MSA',
            'percent': 1,
        },
        {
            'market': 'North Port-Sarasota-Bradenton, FL MSA',
            'percent': 0.5,
        },
    ],
    'Toronto Maple Leafs': [
        {
            'market': 'Greater Toronto and Hamilton Area',
            'percent': 1,
        },
        {
            'market': 'Kitchener, Ontario',
            'percent': 0.5,
        },
        {
            'market': 'London, Ontario',
            'percent': 0.5,
        },
    ],
    'Utah Hockey Club': [
        {
            'market': 'Salt Lake City, UT MSA',
            'percent': 1,
        },
        {
            'market': 'Provo-Orem, UT MSA',
            'percent': 0.5,
        },
        {
            'market': 'Ogden-Clearfield, UT MSA',
            'percent': 0.5,
        },
        {
            'market': 'Logan, UT-ID MSA',
            'percent': 0.5,
        },
    ],
    'Vancouver Canucks': [
        {
            'market': 'Greater Vancouver',
            'percent': 1,
        },
        {
            'market': 'Victoria, British Columbia',
            'percent': 0.5,
        },
        {
            'market': 'Kelowna',
            'percent': 0.5,
        },
    ],
    'Vegas Golden Knights': [
        {
            'market': 'Las Vegas-Henderson-Paradise, NV MSA',
            'percent': 1,
        },
        {
            'market': 'Reno, NV MSA',
            'percent': 0.5,
        },
        {
            'market': 'St. George, UT MSA',
            'percent': 0.5,
        },
    ],
    'Washington Capitals': [
        {
            'market': 'Washington-Arlington-Alexandria, DC-VA-MD-WV MSA',
            'percent': 1,
        },
        {
            'market': 'Baltimore-Columbia-Towson, MD MSA',
            'percent': 0.5,
        },
        {
            'market': 'Richmond, VA MSA',
            'percent': 0.5,
        },
    ],
    'Winnipeg Jets': [
        {
            'market': 'Winnipeg Metropolitan Region',
            'percent': 1,
        },
        {
            'market': 'Baltimore-Columbia-Towson, MD MSA',
            'percent': 0.5,
        },
        {
            'market': 'Richmond, VA MSA',
            'percent': 0.5,
        },
        {
            'market': 'Regina, Saskatchewan',
            'percent': 0.3,
        },
        {
            'market': 'Saskatoon',
            'percent': 0.3,
        },
    ],
}

In [5]:
team_markets = pl.DataFrame([
    {
        'team': team,
        'city': market['market'],
        'percent': market['percent'],
    }
    for team, markets in market_maps.items()
    for market in markets
])

In [6]:
(
    team_markets
    .join(
        cities,
        on='city',
        how='left'
    )
    .with_columns(
        (pl.col('gdp') * pl.col('percent')).alias('tru_gdp'),
        (pl.col('population') * pl.col('percent')).alias('tru_population')
    )
    .group_by('team')
    .agg(
        pl.sum('tru_gdp').alias('tru_gdp'),
        pl.sum('tru_population').alias('tru_population')
    )
    .to_pandas()
)

,team,tru_gdp,tru_population
0,Tampa Bay Lightning,244.0985,3736435.5
1,Utah Hockey Club,179.3780,2058292.0
2,Chicago Blackhawks,924.9070,11113802.5
3,Buffalo Sabres,153.3455,2247124.5
4,Pittsburgh Penguins,223.0310,3103119.0
5,St. Louis Blues,223.1690,3011961.5
6,Minnesota Wild,356.2345,4054361.0
7,Toronto Maple Leafs,371.8745,7853695.0
8,Dallas Stars,707.5960,8278154.0
9,New York Islanders,803.6003,6350612.4


In [130]:
(
    cities
    .filter(
        (pl.col('city').str.contains('UT')) & 
        (pl.col('country').str.contains(''))
    )
    .to_pandas()
)

,city,country,gdp,population
0,"Salt Lake City, UT MSA",United States,135.409,1266191.0
1,"Provo-Orem, UT MSA",United States,40.761,715001.0
2,"Ogden-Clearfield, UT MSA",United States,39.168,713839.0
3,"St. George, UT MSA",United States,9.561,197680.0
4,"Logan, UT-ID MSA",United States,8.009,155362.0
